# 📝 Limpieza Robusta y Preparación de Datos (Data Pre-processing)
El objetivo de este módulo es garantizar la integridad estructural de nuestra serie de tiempo para que esté lista para el análisis exploratorio y el modelado. Para lograrlo, hemos encapsulado todos los pasos críticos en una única función de sanitización.

## 1. Función Central: sanitizar_serie_tiempo()
Esta función actúa como una "caja negra" que toma los datos crudos, corrige los errores de formato, y se asegura de que la serie de tiempo sea continua y sin duplicados en el índice.

### 1.1 Definición de la Función de Sanitización

In [7]:
import pandas as pd
import numpy as np 

def sanitizar_serie_tiempo(df, columna_fecha='Fecha', frecuencia_esperada='D'):
    """
    Realiza la limpieza estructural y formal completa, corrigiendo el error de Key Error en el índice.
    """
    print("--- INICIO DEL PROCESO DE SANITIZACIÓN ---")
    
    df_sanitizado = df.copy() 
    
    # --- 1. Manejo de Errores Críticos (Estructura) ---
    try:
        # A. Verificar existencia y renombrar columna de fecha
        if columna_fecha not in df_sanitizado.columns:
            raise ValueError(f"❌ Error Crítico: La columna de fecha '{columna_fecha}' no se encuentra.")
        
        df_sanitizado = df_sanitizado.rename(columns={columna_fecha: 'Fecha'})

        # B. Transformación a datetime 
        df_sanitizado['Fecha'] = pd.to_datetime(df_sanitizado['Fecha'], errors='coerce')
        
        # C. ¡CORRECCIÓN! Eliminar las filas con fechas no válidas ANTES de establecer el índice.
        # Usamos 'Fecha' como columna, que aún existe.
        nan_fechas_antes = df_sanitizado['Fecha'].isna().sum()
        if nan_fechas_antes > 0:
            print(f"⚠️ Advertencia: Eliminando {nan_fechas_antes} filas con fechas no convertibles.")
            df_sanitizado.dropna(subset=['Fecha'], inplace=True)
            
        # D. Establecimiento del índice
        df_sanitizado = df_sanitizado.set_index('Fecha').sort_index()

    except Exception as e:
        # En caso de que falle la carga o el renombramiento
        print(f"❌ ERROR FATAL en el manejo de tipos o índice: {e}")
        return None 
        
    # --- 2. Coerción de Tipos de Datos y Duplicados ---
    
    for col in df_sanitizado.columns:
        df_sanitizado[col] = pd.to_numeric(df_sanitizado[col], errors='coerce')

    duplicados = df_sanitizado.index.duplicated().sum()
    if duplicados > 0:
        print(f"⚠️ Advertencia: Eliminando {duplicados} entradas duplicadas en el índice.")
        df_sanitizado = df_sanitizado[~df_sanitizado.index.duplicated(keep='first')]
    else:
        print("✅ Índice libre de duplicados.")

    # --- 3. Verificación de Continuidad (Granularidad) ---
    idx_completo = pd.date_range(start=df_sanitizado.index.min(), 
                                 end=df_sanitizado.index.max(), 
                                 freq=frecuencia_esperada)
    
    fechas_faltantes = idx_completo.difference(df_sanitizado.index)
    
    if len(fechas_faltantes) > 0:
        print(f"⚠️ Advertencia: Se detectaron {len(fechas_faltantes)} días faltantes (huecos).")
        df_sanitizado = df_sanitizado.reindex(idx_completo)
    else:
        print(f"✅ La serie es continua con frecuencia '{frecuencia_esperada}'.")
        
    # --- 4. Reporte Final ---
    nulos_totales = df_sanitizado.isnull().sum().sum()
    print(f"\n✨ RESUMEN DE SANITIZACIÓN: La serie limpia tiene {nulos_totales} NaNs para imputar.")
    print("--- PROCESO DE SANITIZACIÓN FINALIZADO ---")
    
    return df_sanitizado

### 1.2 Ejecución de la Limpieza (Flujo Central)

Aquí cargaremos nuestras librerías y ejecutaremos el código de nuestra función de sanitización tras la carga de nuestros datos

In [8]:
# Cargar librerías
import pandas as pd

# Cargar el archivo CSV
df_crudo = pd.read_csv("dataset/pinar_del_rio_temperaturas_2024_2025.csv" )

print("Datos crudos cargados, listos para sanitización.")

Datos crudos cargados, listos para sanitización.


In [ ]:
# Ejecutar la función de sanitización "todo en uno"
df_limpio = sanitizar_serie_tiempo(df_crudo, columna_fecha='Fecha')

if df_limpio is not None:
    print("\nDataFrame Limpio (Primeras filas):")
    print(df_limpio.head())
    print("\nTipos de Datos del DataFrame Limpio:")
    print(df_limpio.info())

    # Guardar el archivo limpio para el siguiente notebook
    df_limpio.to_csv("dataset/pinar_del_rio_temperaturas_LIMPIO_FINAL.csv")

--- INICIO DEL PROCESO DE SANITIZACIÓN ---
✅ Índice libre de duplicados.
✅ La serie es continua con frecuencia 'D'.

✨ RESUMEN DE SANITIZACIÓN: La serie limpia tiene 0 NaNs para imputar.
--- PROCESO DE SANITIZACIÓN FINALIZADO ---

DataFrame Limpio (Primeras filas):
            Temp_Maxima_C  Temp_Minima_C
Fecha                                   
2024-01-01           24.1           13.1
2024-01-02           24.2           13.7
2024-01-03           23.2           15.0
2024-01-04           27.2           15.6
2024-01-05           25.8           17.0

Tipos de Datos del DataFrame Limpio:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 671 entries, 2024-01-01 to 2025-11-01
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Temp_Maxima_C  671 non-null    float64
 1   Temp_Minima_C  671 non-null    float64
dtypes: float64(2)
memory usage: 15.7 KB
None
